# **Perspectiva General de Mercado - DatAlpine**
*Filtro de tipo de mercado*

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_")  and "puebla" in nombre and '2024' in nombre
    and 'q' not in nombre
    and 'financiamientos' not in nombre
    and 'poblacion' not in nombre
    and 'salarios' not in nombre
    and 'publicacion' not in nombre
    
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_puebla',
 'df_alfa_febrero_2024_puebla',
 'df_alfa_julio_2024_puebla',
 'df_alfa_junio_2024_puebla',
 'df_alfa_septiembre_2024_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Propiedad':'propiedad','Precio':'precio','Metros_total':'metros_total','m_total':'metros_total','m_construido':'metros_construido','Metros_construido':'metros_construido','Estacionamientos':'estacionamiento','Recamaras':'recamaras','Banos':'baño','Medio_banos':'medio_baño','Banos_Total':'baño_total','Precio_M2_Construido':'precio_m2_construido'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['categoria','Tipo','propiedad','precio','metros_total','metros_construido', 'precio_m2_construido','estacionamiento','recamaras', 'baño', 'medio_baño', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [4]:
df_total = pd.concat(dataframes_list, ignore_index=True)

In [5]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27934 entries, 0 to 27933
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   categoria             27934 non-null  object 
 1   Tipo                  27934 non-null  object 
 2   propiedad             27934 non-null  object 
 3   precio                27934 non-null  float64
 4   metros_total          27934 non-null  int64  
 5   metros_construido     27934 non-null  int64  
 6   precio_m2_construido  27934 non-null  float64
 7   estacionamiento       27934 non-null  int64  
 8   recamaras             27934 non-null  int64  
 9   baño                  27934 non-null  int64  
 10  medio_baño            27934 non-null  int64  
 11  baño_total            27934 non-null  float64
dtypes: float64(3), int64(6), object(3)
memory usage: 2.6+ MB


In [29]:
# Definición de rangos de precio
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500001, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),
    "B2": (2750001, 3000000),
    "B3": (3000001, 3250000),
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf')),
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio <= limite_superior:  # Cambiado a <=
            return categoria
    return None  # Retornar None para manejarlo después

# Limpieza del DataFrame
df_total = df_total.drop(columns=['categoria'], errors='ignore')

# Convertir 'precio' a numérico y manejar errores
df_total['precio'] = pd.to_numeric(df_total['precio'], errors='coerce')
df_total = df_total.dropna(subset=['precio'])  # Eliminar filas con precios NaN

# Asignar categorías
df_total['categoria'] = df_total['precio'].apply(asignar_categoria)

# Verificar categorías inválidas
categorias_invalidas = df_total[df_total['categoria'].isnull()]
if not categorias_invalidas.empty:
    print("Precios sin categoría:")
    print(categorias_invalidas[['precio']])

In [30]:
df_total.loc[:, 'segmento'] = df_total['categoria'].apply(lambda x: 'ELITE' if x == 'ELITE' else x[0])

In [31]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 

# Limitar los valores  
df_total['baño_total'] = df_total['baño_total'].clip(lower=1) #,upper=10
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1)#upper=8 

df_total = df_total[df_total['precio'] != 0]
df_total = df_total[df_total['metros_total'] != 0]
df_total = df_total[df_total['metros_construido'] != 0]

#df_total = df_total[~df_total['categoria'].str.contains('L')]

In [65]:
df_total = df_total[df_total['metros_total'] != 220000]
df_total.loc[df_total['metros_total'] == 19124, 'metros_total'] = 192

df_total.loc[df_total['metros_total'] == 60003, 'metros_total'] = 600
df_total.loc[df_total['metros_total'] == 58135, 'metros_total'] = 581

In [39]:
df_total['Tipo'].unique()

array(['Casa', 'Departamento'], dtype=object)

In [66]:
casa_df = df_total[df_total['Tipo'].isin(['Casa', 'Casa En Fraccionamiento', 'Casa En Condominio', 'Casa Duplex'])]
depa_df = df_total[df_total['Tipo'].isin(['Departamento','Duplex','Condominio Horizontal'])]

# Casas

### Revisión de m2

In [85]:
casa_df = casa_df[casa_df['metros_total'] > 40]
casa_df = casa_df[casa_df['metros_construido'] > 40]

casa_df = casa_df[casa_df['metros_construido'] < 1169]
casa_df = casa_df[casa_df['metros_total'] < 2400]



In [68]:

printtt = df_total[df_total['metros_total'] == 19124]
printtt


,Tipo,propiedad,precio,metros_total,metros_construido,precio_m2_construido,estacionamiento,recamaras,baño,medio_baño,baño_total,segmento,categoria


In [86]:
# metros_total en casas
precios = casa_df['metros_total'].unique()
precios = np.sort(precios)
precios

array([  41,   43,   45,   47,   48,   50,   51,   52,   53,   54,   55,
         56,   57,   58,   59,   60,   61,   62,   63,   64,   65,   66,
         67,   68,   69,   70,   71,   72,   73,   74,   75,   76,   77,
         78,   79,   80,   81,   82,   83,   84,   85,   86,   87,   88,
         89,   90,   91,   92,   93,   94,   95,   96,   97,   98,   99,
        100,  101,  102,  103,  104,  105,  106,  107,  108,  109,  110,
        111,  112,  113,  114,  115,  116,  117,  118,  119,  120,  121,
        122,  123,  124,  125,  126,  127,  128,  129,  130,  131,  132,
        133,  134,  135,  136,  137,  138,  139,  140,  141,  142,  143,
        144,  145,  146,  147,  148,  149,  150,  151,  152,  153,  154,
        155,  156,  158,  159,  160,  161,  162,  163,  164,  165,  166,
        167,  168,  169,  170,  171,  172,  173,  174,  175,  176,  177,
        178,  179,  180,  181,  182,  183,  184,  185,  186,  187,  188,
        189,  190,  191,  192,  193,  194,  195,  1

In [87]:
# metros_construido en casas
precios = casa_df['metros_construido'].unique()
precios = np.sort(precios)
precios

array([  41,   42,   44,   45,   47,   48,   49,   50,   51,   52,   54,
         55,   56,   57,   58,   59,   60,   61,   62,   63,   64,   65,
         66,   67,   68,   69,   70,   71,   72,   73,   74,   75,   76,
         77,   78,   79,   80,   81,   82,   83,   84,   85,   86,   87,
         88,   89,   90,   91,   92,   93,   94,   95,   96,   97,   98,
         99,  100,  101,  102,  103,  104,  105,  106,  107,  108,  109,
        110,  111,  112,  113,  114,  115,  116,  117,  118,  119,  120,
        121,  122,  123,  124,  125,  126,  127,  128,  129,  130,  131,
        132,  133,  134,  135,  136,  137,  138,  139,  140,  141,  142,
        143,  144,  145,  146,  147,  148,  149,  150,  151,  152,  153,
        154,  155,  156,  157,  158,  159,  160,  161,  162,  163,  164,
        165,  166,  167,  168,  169,  170,  171,  172,  173,  174,  175,
        176,  177,  178,  179,  180,  181,  182,  183,  184,  185,  186,
        187,  188,  189,  190,  191,  192,  193,  1

In [92]:
# metros_total en casas cuando segmento es 'B'
precios = casa_df[casa_df['segmento'] == 'E']['metros_total'].unique()
#precios = casa_df[casa_df['segmento'] == 'S']['metros_total'].value_counts()
precios = np.sort(precios)
precios

array([  41,   45,   47,   48,   50,   51,   52,   53,   54,   55,   56,
         57,   58,   59,   60,   61,   62,   63,   64,   65,   66,   67,
         68,   69,   70,   71,   72,   73,   74,   75,   76,   78,   79,
         80,   81,   82,   83,   84,   85,   86,   87,   88,   89,   90,
         92,   93,   94,   95,   96,   97,   98,   99,  100,  101,  102,
        103,  104,  105,  106,  108,  109,  110,  111,  112,  113,  114,
        115,  116,  117,  118,  119,  120,  121,  122,  123,  125,  126,
        127,  128,  130,  131,  132,  133,  134,  135,  140,  141,  142,
        143,  144,  145,  146,  149,  150,  151,  152,  153,  154,  155,
        156,  158,  160,  161,  163,  164,  165,  167,  168,  170,  171,
        172,  173,  174,  175,  177,  179,  180,  181,  182,  184,  185,
        186,  188,  189,  190,  191,  192,  194,  195,  196,  198,  199,
        200,  201,  202,  203,  204,  205,  208,  210,  211,  212,  213,
        214,  215,  216,  218,  219,  220,  221,  2

In [90]:
casa_df = casa_df[~((casa_df['segmento'] == 'S') & (casa_df['metros_total'] == 45))]
casa_df = casa_df[~((casa_df['segmento'] == 'C') & (casa_df['metros_total'] == 41))]

casa_df = casa_df[~((casa_df['segmento'] == 'C') & (casa_df['metros_total'].isin([9865, 11410, 11539, 12262, 12460, 16887])))]


## Casa promedio

In [14]:
#casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0]
grupo_promedios = round(casa_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(casa_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

#print(grupo_promedios)
#print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
ELITE                  5.0        5.0    4.0
L                      4.0        4.0    3.0
S                      3.0        4.0    3.0
general                2.0        3.0    2.0


## Precio Mercado

In [45]:
#casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S','L','ELITE']#, 
grupo_descripcion_casa = casa_df.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,260000.0,1000000.0,6.795821e+05
1,D,1005000.0,1750000.0,1.352932e+06
2,C,1754500.0,2500000.0,2.134139e+06
3,B,2505000.0,3250000.0,2.884991e+06
4,A,3260000.0,4000000.0,3.676641e+06
5,S,4018904.0,12000000.0,6.627364e+06
6,L,12250000.0,22000000.0,1.555534e+07
7,ELITE,22500000.0,135000000.0,3.297929e+07


In [16]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17808\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,260,000.00
1     $2,505,000.00
2     $1,754,500.00
3     $1,005,000.00
4       $260,000.00
5    $22,500,000.00
6    $12,250,000.00
7     $4,018,904.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17808\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      $4,000,000.00
1      $3,250,000.00
2      $2,500,000.00
3      $1,750,000.00
4      $1,000,000.00
5    $335,410,000.00
6     $22,000,000.00
7     $12,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please 

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,260,000.00","$4,000,000.00","$3,676,641.24"
1,B,"$2,505,000.00","$3,250,000.00","$2,884,990.84"
2,C,"$1,754,500.00","$2,500,000.00","$2,134,139.26"
3,D,"$1,005,000.00","$1,750,000.00","$1,353,098.23"
4,E,"$260,000.00","$1,000,000.00","$679,558.28"
5,ELITE,"$22,500,000.00","$335,410,000.00","$34,968,964.19"
6,L,"$12,250,000.00","$22,000,000.00","$15,555,342.04"
7,S,"$4,018,904.00","$12,000,000.00","$6,627,363.62"


In [31]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [17]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['precio'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

             Mínimo           Máximo       Promedio segmento
precio  $260,000.00  $335,410,000.00  $4,087,659.35  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_17808\2555968983.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $260,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17808\2555968983.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $335,410,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17808\2555968983.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will ra

In [18]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [19]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento          Mínimo           Máximo        Promedio
0        A   $3,260,000.00    $4,000,000.00   $3,676,641.24
1        B   $2,505,000.00    $3,250,000.00   $2,884,990.84
2        C   $1,754,500.00    $2,500,000.00   $2,134,139.26
3        D   $1,005,000.00    $1,750,000.00   $1,353,098.23
4        E     $260,000.00    $1,000,000.00     $679,558.28
5    ELITE  $22,500,000.00  $335,410,000.00  $34,968,964.19
6        L  $12,250,000.00   $22,000,000.00  $15,555,342.04
7        S   $4,018,904.00   $12,000,000.00   $6,627,363.62
8  general     $260,000.00  $335,410,000.00   $4,087,659.35


In [20]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S','L','ELITE','general']
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,"$260,000.00","$1,000,000.00","$679,558.28"
1,D,"$1,005,000.00","$1,750,000.00","$1,353,098.23"
2,C,"$1,754,500.00","$2,500,000.00","$2,134,139.26"
3,B,"$2,505,000.00","$3,250,000.00","$2,884,990.84"
4,A,"$3,260,000.00","$4,000,000.00","$3,676,641.24"
5,S,"$4,018,904.00","$12,000,000.00","$6,627,363.62"
6,L,"$12,250,000.00","$22,000,000.00","$15,555,342.04"
7,ELITE,"$22,500,000.00","$335,410,000.00","$34,968,964.19"
8,general,"$260,000.00","$335,410,000.00","$4,087,659.35"


In [22]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Terreno

In [93]:
#casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','ELITE']#,
grupo_descripcion_casa = casa_df.groupby('segmento')['metros_total'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Calcular el percentil 95%
#percentiles_95 = casa_df.groupby('segmento')['metros_total'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'metros_total': 'Máximo_95%'}, inplace=True)
# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion_casa.merge(percentiles_95, on='segmento')
# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True) # Eliminar la columna temporal


# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,41.0,1803.0,172.340533
1,D,43.0,780.0,142.325374
2,C,45.0,1200.0,134.843647
3,B,75.0,780.0,157.875564
4,A,62.0,905.0,181.111590
5,S,74.0,2292.0,370.573165
6,L,80.0,2178.0,732.942495
7,ELITE,260.0,2300.0,1160.381643


In [76]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17808\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     62.00 m²
1     75.00 m²
2     45.00 m²
3     43.00 m²
4     41.00 m²
5    260.00 m²
6     80.00 m²
7     74.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17808\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    22,161.00 m²
1    26,964.00 m²
2    16,887.00 m²
3       780.00 m²
4     7,367.00 m²
5    20,443.00 m²
6    12,030.00 m²
7    49,085.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripc

,segmento,Mínimo,Máximo,Promedio
0,A,62.00 m²,"22,161.00 m²",212.65 m²
1,B,75.00 m²,"26,964.00 m²",232.68 m²
2,C,45.00 m²,"16,887.00 m²",266.83 m²
3,D,43.00 m²,780.00 m²,142.33 m²
4,E,41.00 m²,"7,367.00 m²",174.93 m²
5,ELITE,260.00 m²,"20,443.00 m²","1,787.41 m²"
6,L,80.00 m²,"12,030.00 m²",869.50 m²
7,S,74.00 m²,"49,085.00 m²",626.35 m²


In [77]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [78]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['metros_total'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                Mínimo        Máximo   Promedio segmento
metros_total  41.00 m²  49,085.00 m²  353.62 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_17808\1147424691.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    41.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17808\1147424691.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    49,085.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17808\1147424691.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and w

In [79]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [80]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo        Máximo     Promedio
0        A   62.00 m²  22,161.00 m²    212.65 m²
1        B   75.00 m²  26,964.00 m²    232.68 m²
2        C   45.00 m²  16,887.00 m²    266.83 m²
3        D   43.00 m²     780.00 m²    142.33 m²
4        E   41.00 m²   7,367.00 m²    174.93 m²
5    ELITE  260.00 m²  20,443.00 m²  1,787.41 m²
6        L   80.00 m²  12,030.00 m²    869.50 m²
7        S   74.00 m²  49,085.00 m²    626.35 m²
8  general   41.00 m²  49,085.00 m²    353.62 m²


In [81]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,41.00 m²,"7,367.00 m²",174.93 m²
1,D,43.00 m²,780.00 m²,142.33 m²
2,C,45.00 m²,"16,887.00 m²",266.83 m²
3,B,75.00 m²,"26,964.00 m²",232.68 m²
4,A,62.00 m²,"22,161.00 m²",212.65 m²
5,S,74.00 m²,"49,085.00 m²",626.35 m²
6,L,80.00 m²,"12,030.00 m²",869.50 m²
7,general,41.00 m²,"49,085.00 m²",353.62 m²


In [44]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_terreno_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Construcción

In [45]:
casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S','L']#, 
grupo_descripcion_casa = casa_df.groupby('segmento')['metros_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,38.0,220000.0,324.764137
1,D,34.0,780.0,158.823671
2,C,41.0,40642.0,307.375152
3,B,63.0,11614.0,225.609476
4,A,62.0,12025.0,241.359637
5,S,74.0,85571.0,661.720425
6,L,120.0,6000.0,643.024150


In [46]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19976\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     62.00 m²
1     63.00 m²
2     41.00 m²
3     34.00 m²
4     38.00 m²
5    120.00 m²
6     74.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19976\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     12,025.00 m²
1     11,614.00 m²
2     40,642.00 m²
3        780.00 m²
4    220,000.00 m²
5      6,000.00 m²
6     85,571.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grup

,segmento,Mínimo,Máximo,Promedio
0,A,62.00 m²,"12,025.00 m²",241.36 m²
1,B,63.00 m²,"11,614.00 m²",225.61 m²
2,C,41.00 m²,"40,642.00 m²",307.38 m²
3,D,34.00 m²,780.00 m²,158.82 m²
4,E,38.00 m²,"220,000.00 m²",324.76 m²
5,L,120.00 m²,"6,000.00 m²",643.02 m²
6,S,74.00 m²,"85,571.00 m²",661.72 m²


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [47]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['metros_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                     Mínimo         Máximo   Promedio segmento
metros_construido  34.00 m²  220,000.00 m²  377.04 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19976\2239152448.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19976\2239152448.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    220,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19976\2239152448.py:24: FutureWarning: Setting an item of incompatible dtype is depre

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [48]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo         Máximo   Promedio
0        A   62.00 m²   12,025.00 m²  241.36 m²
1        B   63.00 m²   11,614.00 m²  225.61 m²
2        C   41.00 m²   40,642.00 m²  307.38 m²
3        D   34.00 m²      780.00 m²  158.82 m²
4        E   38.00 m²  220,000.00 m²  324.76 m²
5        L  120.00 m²    6,000.00 m²  643.02 m²
6        S   74.00 m²   85,571.00 m²  661.72 m²
7  general   34.00 m²  220,000.00 m²  377.04 m²


In [49]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,38.00 m²,"220,000.00 m²",324.76 m²
1,D,34.00 m²,780.00 m²,158.82 m²
2,C,41.00 m²,"40,642.00 m²",307.38 m²
3,B,63.00 m²,"11,614.00 m²",225.61 m²
4,A,62.00 m²,"12,025.00 m²",241.36 m²
5,S,74.00 m²,"85,571.00 m²",661.72 m²
6,L,120.00 m²,"6,000.00 m²",643.02 m²
7,general,34.00 m²,"220,000.00 m²",377.04 m²


In [50]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_construidos_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## Pecio M2 Construcción

In [51]:
casa_df['precio_m2_construido']

0         5957.45
1         4000.00
2         1500.00
3         3596.18
4         1547.14
           ...   
27927    49566.29
27929    52204.18
27930    39539.90
27931    26086.96
27932    67500.00
Name: precio_m2_construido, Length: 20657, dtype: float64

In [52]:
casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L']#
grupo_descripcion_casa = casa_df.groupby('segmento')['precio_m2_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,128.06,112307.69,6295.080900
1,D,1577.08,51470.59,10949.728732
2,C,44.29,47560.98,15208.544791
3,B,221.07,42063.49,16445.397471
4,A,303.53,56451.61,17886.297575
5,S,93.49,121621.62,18770.432546
6,L,3333.33,165833.33,28192.129580


In [53]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19976\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      $303.53
1      $221.07
2       $44.29
3    $1,577.08
4      $128.06
5    $3,333.33
6       $93.49
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19976\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $56,451.61
1     $42,063.49
2     $47,560.98
3     $51,470.59
4    $112,307.69
5    $165,833.33
6    $121,621.62
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_cas

,segmento,Mínimo,Máximo,Promedio
0,A,$303.53,"$56,451.61","$17,886.30"
1,B,$221.07,"$42,063.49","$16,445.40"
2,C,$44.29,"$47,560.98","$15,208.54"
3,D,"$1,577.08","$51,470.59","$10,949.73"
4,E,$128.06,"$112,307.69","$6,295.08"
5,L,"$3,333.33","$165,833.33","$28,192.13"
6,S,$93.49,"$121,621.62","$18,770.43"


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [54]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['precio_m2_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                      Mínimo       Máximo    Promedio segmento
precio_m2_construido  $44.29  $165,833.33  $14,043.52  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19976\155160619.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $44.29
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19976\155160619.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $165,833.33
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19976\155160619.py:23: FutureWarning: Setting an item of incompatible dtype is deprec

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [55]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo       Máximo    Promedio
0        A    $303.53   $56,451.61  $17,886.30
1        B    $221.07   $42,063.49  $16,445.40
2        C     $44.29   $47,560.98  $15,208.54
3        D  $1,577.08   $51,470.59  $10,949.73
4        E    $128.06  $112,307.69   $6,295.08
5        L  $3,333.33  $165,833.33  $28,192.13
6        S     $93.49  $121,621.62  $18,770.43
7  general     $44.29  $165,833.33  $14,043.52


In [56]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,$128.06,"$112,307.69","$6,295.08"
1,D,"$1,577.08","$51,470.59","$10,949.73"
2,C,$44.29,"$47,560.98","$15,208.54"
3,B,$221.07,"$42,063.49","$16,445.40"
4,A,$303.53,"$56,451.61","$17,886.30"
5,S,$93.49,"$121,621.62","$18,770.43"
6,L,"$3,333.33","$165,833.33","$28,192.13"
7,general,$44.29,"$165,833.33","$14,043.52"


In [57]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

# _____
_____
# ANTES

In [7]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500001, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),
    "B2": (2750001, 3000000),
    "B3": (3000001, 3250000),
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf')),
}

# Función para asignar la categoría según el precio
#def asignar_categoria(precio):
#    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
#        if limite_inferior <= precio < limite_superior:
#            return categoria
#    return None  # En caso de que el precio no caiga en ningún rango (caso raro)
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return "Sin categoría"  # Asignar una etiqueta clara en lugar de None
df_total = df_total.drop(columns=['categoria'], errors='ignore')


# Convertir 'precio' a numérico y manejar errores para tener la categoría correcta 
df_total['precio'] = pd.to_numeric(df_total['precio'], errors='coerce')
# Llenar valores faltantes con un número por defecto, por ejemplo, 0
df_total['precio'] = df_total['precio'].fillna(0).astype(int)
df_total = df_total[df_total['precio'] > 0]


#df_total['categoria'] = df_total['precio'].apply(asignar_categoria)
df_total['categoria'] = df_total['precio'].apply(asignar_categoria)
categorias_invalidas = df_total[df_total['categoria'].isnull()]
if not categorias_invalidas.empty:
    print("Precios sin categoría:")
    print(categorias_invalidas[['precio']])


In [8]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 
df_total['baño_total'] = df_total['baño_total'].clip(lower=1,upper=10) # Limitar los valores 
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1,)#Mínimo 1 #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1,)#upper=8 

df_total = df_total[df_total['precio'] != 0]
#df_total = df_total[df_total['precio'] > 4800]
df_total = df_total[df_total['metros_total'] > 20]
df_total = df_total[df_total['metros_construido'] > 20]

- Casa promedio
- Casa Mediana
- Precio mercado

- m2 Terreno
- M2 Construcción
- Precio m2 construcción

In [10]:
casas_df = df_total[df_total['Tipo'].isin(['Casa', 'Casa En Fraccionamiento', 'Casa En Condominio', 'Casa Duplex'])]
departamentos_df = df_total[df_total['Tipo'].isin(['Departamento','Duplex','Condominio Horizontal'])]

## Casa promedio

In [11]:
casas_df.loc[:, 'segmento'] = casas_df['categoria'].str[0]
grupo_promedios = round(casas_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(casas_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      4.0        4.0    3.0
S                      3.0        4.0    3.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      4.0        4.0    3.0
S                      3.0        4.0    3.0
general                2.0        3.0    2.0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\2149301391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  casas_df.loc[:, 'segmento'] = casas_df['categoria'].str[0]


## Depa Promedio

In [12]:
departamentos_df.loc[:, 'segmento'] = departamentos_df['categoria'].str[0]
grupo_promedios = round(departamentos_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(departamentos_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_depa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        2.0    2.0
B                      2.0        2.0    2.0
C                      2.0        2.0    2.0
D                      2.0        2.0    2.0
E                      1.0        2.0    1.0
L                      3.0        3.0    3.0
S                      2.0        3.0    2.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          2      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        2.0    2.0
B                      2.0        2.0    2.0
C                      2.0        2.0    2.0
D                      2.0        2.0    2.0
E                      1.0        2.0    1.0
L                      3.0        3.0    3.0
S                      2.0        3.0    2.0
general                2.0        2.0    2.0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\3929789102.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  departamentos_df.loc[:, 'segmento'] = departamentos_df['categoria'].str[0]


## Precio Mercado

In [13]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion = grupo_descripcion[['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
grupo_descripcion 

,segmento,Mínimo,Máximo,Promedio
0,A,3252000.0,3999000.0,3.645082e+06
1,B,2502235.0,3249000.0,2.879851e+06
2,C,1754500.0,2499000.0,2.135168e+06
3,D,1005000.0,1749000.0,1.380241e+06
4,E,148640.0,335410000.0,2.477425e+06
5,L,12250000.0,21995000.0,1.546130e+07
6,S,500000.0,22000000.0,5.692560e+06


In [14]:

# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
grupo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
print(grupo_descripcion)


  segmento      Mínimo      Máximo      Promedio
0        A   3252000.0   3950000.0  3.645082e+06
1        B   2502235.0   3200000.0  2.879851e+06
2        C   1754500.0   2450000.0  2.135168e+06
3        D   1005000.0   1690000.0  1.380241e+06
4        E    148640.0    998000.0  2.477425e+06
5        L  12250000.0  20000000.0  1.546130e+07
6        S    500000.0  11000000.0  5.692560e+06


In [15]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\1329763181.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,252,000.00
1     $2,502,235.00
2     $1,754,500.00
3     $1,005,000.00
4       $148,640.00
5    $12,250,000.00
6       $500,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\1329763181.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,950,000.00
1     $3,200,000.00
2     $2,450,000.00
3     $1,690,000.00
4       $998,000.00
5    $20,000,000.00
6    $11,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descr

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,252,000.00","$3,950,000.00","$3,645,082.27"
1,B,"$2,502,235.00","$3,200,000.00","$2,879,851.13"
2,C,"$1,754,500.00","$2,450,000.00","$2,135,167.59"
3,D,"$1,005,000.00","$1,690,000.00","$1,380,240.97"
4,E,"$148,640.00","$998,000.00","$2,477,425.48"
5,L,"$12,250,000.00","$20,000,000.00","$15,461,298.26"
6,S,"$500,000.00","$11,000,000.00","$5,692,559.82"


In [16]:
import pandas as pd

# Supongamos que df_total ya está definido y contiene las columnas 'metros_construido' y 'precio_m2_construido'.

# Filtrar los datos según las condiciones especificadas
filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Paso 1: Calcular las estadísticas descriptivas
todo_descripcion = filtrado_df['precio_m2_construido'].describe()[['min', 'max', 'mean']].to_frame().T

# Agregar la columna de segmento
todo_descripcion['segmento'] = 'general'

# Paso 2: Renombrar las columnas
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Paso 3: Calcular el percentil 95%
percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)

# Paso 4: Reemplazar el valor máximo con el percentil 95%
todo_descripcion['Máximo'] = percentil_95

# Paso 5: Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

# Mostrar el resultado final
print(todo_descripcion)


                           Mínimo       Máximo     Promedio segmento
precio_m2_construido  $315,121.95  $792,667.08  $675,258.04  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\79234579.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $315,121.95
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\79234579.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $792,667.08
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\79234579.py:32: FutureWarning: Setting an item of incompatible dtype is depr

In [17]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [18]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,252,000.00","$3,950,000.00","$3,645,082.27"
1,B,"$2,502,235.00","$3,200,000.00","$2,879,851.13"
2,C,"$1,754,500.00","$2,450,000.00","$2,135,167.59"
3,D,"$1,005,000.00","$1,690,000.00","$1,380,240.97"
4,E,"$148,640.00","$998,000.00","$2,477,425.48"
5,L,"$12,250,000.00","$20,000,000.00","$15,461,298.26"
6,S,"$500,000.00","$11,000,000.00","$5,692,559.82"
7,general,"$315,121.95","$792,667.08","$675,258.04"


In [16]:
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]

# Función para limpiar valores con unidades y convertirlos a float
def limpiar_valor(valor):
    # Eliminar unidades (m², $, etc.) y convertir a float
    if isinstance(valor, str):
        return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))
    else:
        return valor

# Aplicar limpieza de valores en las columnas que contienen medidas
for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)
# Función para formatear nuevamente los valores en el formato deseado
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Volver a aplicar el formato monetario después de los ajustes
for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)



In [17]:
# Limpiar valores y convertir a float
def limpiar_valor(valor):
    return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))

for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)

# Verificar el orden de los segmentos
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]
resultados_finales_descripcion = resultados_finales_descripcion.set_index('segmento').reindex(orden_segmentos).reset_index()

# Validar que los valores no sean inconsistentes
for i in range(1, len(resultados_finales_descripcion)):
    for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
        valor_actual = resultados_finales_descripcion.loc[i, columna]
        valor_anterior = resultados_finales_descripcion.loc[i - 1, columna]
        
        # Ajustar valores inconsistentes
        if valor_actual < valor_anterior:
            resultados_finales_descripcion.loc[i, columna] = valor_anterior

# Volver a formatear los valores a formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"

for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)

# Guardar los resultados en un archivo JSON
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='records', force_ascii=False)
with open('assets/tablas/grupo_precio_mercado_ajustado.json', 'w', encoding='utf-8') as f:
    f.write(resultados_finales_desc_json)


In [19]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,252,000.00","$3,950,000.00","$3,645,082.27"
1,B,"$2,502,235.00","$3,200,000.00","$2,879,851.13"
2,C,"$1,754,500.00","$2,450,000.00","$2,135,167.59"
3,D,"$1,005,000.00","$1,690,000.00","$1,380,240.97"
4,E,"$148,640.00","$998,000.00","$2,477,425.48"
5,L,"$12,250,000.00","$20,000,000.00","$15,461,298.26"
6,S,"$500,000.00","$11,000,000.00","$5,692,559.82"
7,general,"$315,121.95","$792,667.08","$675,258.04"


## M2 Terreno

In [20]:
df_total = df_total[df_total['metros_construido'] > 40]
df_total_terreno = df_total[~df_total['metros_total'].isin([0, 1, 2])]

df_total_terreno = df_total_terreno[(df_total_terreno['metros_total'] != 1) & (df_total_terreno['metros_total'] > 20)]

df_descripcion = df_total_terreno.groupby('segmento')['metros_total'].describe(percentiles=[0.96]).reset_index()
df_descripcion = df_descripcion[['segmento', 'min', '96%', 'mean']] #Columnas deseadas
df_descripcion.rename(columns={
    'min': 'Mínimo',
    '96%': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\1393368157.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    50.00 m²
1    43.00 m²
2    40.00 m²
3    38.00 m²
4    31.00 m²
5    80.00 m²
6    45.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\1393368157.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      300.00 m²
1      270.40 m²
2      277.00 m²
3      300.00 m²
4      650.56 m²
5    2,178.00 m²
6      878.04 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato

,segmento,Mínimo,Máximo,Promedio
0,A,50.00 m²,300.00 m²,273.29 m²
1,B,43.00 m²,270.40 m²,272.16 m²
2,C,40.00 m²,277.00 m²,277.01 m²
3,D,38.00 m²,300.00 m²,157.23 m²
4,E,31.00 m²,650.56 m²,335.01 m²
5,L,80.00 m²,"2,178.00 m²",837.64 m²
6,S,45.00 m²,878.04 m²,"2,430.26 m²"


In [21]:
todo_descripcion = df_total[~df_total['metros_total'].isin([0, 1, 2])]['metros_total'].describe(percentiles=[0.95])[['min', '95%', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    'min': 'Mínimo',
    '95%': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(3):  # 0, 1, 2 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\1067183333.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    31.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\1067183333.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    700.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\1067183333.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

,Mínimo,Máximo,Promedio,segmento
metros_total,31.00 m²,700.00 m²,905.97 m²,general


In [22]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_terreno.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [23]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,50.00 m²,300.00 m²,273.29 m²
1,B,43.00 m²,270.40 m²,272.16 m²
2,C,40.00 m²,277.00 m²,277.01 m²
3,D,38.00 m²,300.00 m²,157.23 m²
4,E,31.00 m²,650.56 m²,335.01 m²
5,L,80.00 m²,"2,178.00 m²",837.64 m²
6,S,45.00 m²,878.04 m²,"2,430.26 m²"
7,general,31.00 m²,700.00 m²,905.97 m²


## M2 Construcción

In [24]:
df_total_terreno = df_total[(df_total['metros_construido'] > 40) & (~df_total['metros_construido'].isin([0, 1,2,3]))]
df_descripcion = df_total_terreno.groupby('segmento')['metros_construido'].describe(percentiles=[0.95]).reset_index()
df_descripcion = df_descripcion[['segmento', 'min', '95%', 'mean']] #Columnas deseadas
df_descripcion.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\1810280376.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     60.00 m²
1     58.00 m²
2     41.00 m²
3     42.00 m²
4     41.00 m²
5    120.00 m²
6     49.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\1810280376.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      324.00 m²
1      300.00 m²
2      288.25 m²
3      312.00 m²
4      579.00 m²
5    1,189.00 m²
6      667.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(

,segmento,Mínimo,Máximo,Promedio
0,A,60.00 m²,324.00 m²,287.29 m²
1,B,58.00 m²,300.00 m²,257.83 m²
2,C,41.00 m²,288.25 m²,303.98 m²
3,D,42.00 m²,312.00 m²,161.40 m²
4,E,41.00 m²,579.00 m²,322.99 m²
5,L,120.00 m²,"1,189.00 m²",649.59 m²
6,S,49.00 m²,667.00 m²,"2,397.74 m²"


In [25]:
todo_descripcion = df_total[~df_total['metros_construido'].isin([0,1,2,3]) & (df_total['metros_total'] > 3)]['metros_total'].describe(percentiles=[0.95])[['min', '95%', 'mean',]].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Máximo Total', 'Promedio'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\2220166576.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    31.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\2220166576.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    700.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\2220166576.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

,Mínimo,Máximo,Promedio,segmento
metros_total,31.00 m²,700.00 m²,905.97 m²,general


In [26]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [27]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,60.00 m²,324.00 m²,287.29 m²
1,B,58.00 m²,300.00 m²,257.83 m²
2,C,41.00 m²,288.25 m²,303.98 m²
3,D,42.00 m²,312.00 m²,161.40 m²
4,E,41.00 m²,579.00 m²,322.99 m²
5,L,120.00 m²,"1,189.00 m²",649.59 m²
6,S,49.00 m²,667.00 m²,"2,397.74 m²"
7,general,31.00 m²,700.00 m²,905.97 m²


## Pecio M2 Construcción

In [28]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
df_total_terreno = df_total_terreno[df_total_terreno['precio_m2_construido'] > 10]
grupo_filtrado = df_total_terreno.groupby('segmento')['precio_m2_construido'].describe(percentiles=[0.95]).reset_index()
# Mantener solo las columnas que deseas (en este caso, min, 95%, mean)
grupo_filtrado = grupo_filtrado[['segmento', 'min', '95%', 'mean']] #Columnas deseadas
grupo_filtrado.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)

grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\952371004.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      $213.98
1      $205.00
2       $44.29
3      $137.52
4      $128.06
5    $1,399.66
6       $87.83
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\952371004.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $39,175.26
1    $30,945.75
2    $28,571.43
3    $22,649.62
4    $18,404.26
5    $42,944.79
6    $38,888.89
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(format

,segmento,Mínimo,Máximo,Promedio
0,A,$213.98,"$39,175.26","$21,676.53"
1,B,$205.00,"$30,945.75","$19,938.89"
2,C,$44.29,"$28,571.43","$18,000.37"
3,D,$137.52,"$22,649.62","$13,098.89"
4,E,$128.06,"$18,404.26","$7,887.42"
5,L,"$1,399.66","$42,944.79","$28,511.68"
6,S,$87.83,"$38,888.89","$21,086.86"


In [29]:
todo_descripcion = df_total[(df_total['precio_m2_construido'] > 6) & (~df_total['metros_construido'].isin([0, 1,2,3]))]['precio_m2_construido'].describe(percentiles=[0.95])[['min', '95%', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general'
todo_descripcion = todo_descripcion[['segmento', 'min', '95%', 'mean']] #Columnas deseadas

todo_descripcion.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\4241514475.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $44.29
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\4241514475.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $34,642.86
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18612\4241514475.py:13: FutureWarning: Setting an item of incompatible dtype is depr

,segmento,Mínimo,Máximo,Promedio
precio_m2_construido,general,$44.29,"$34,642.86","$16,812.72"


In [30]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_filtrado, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [31]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,$213.98,"$39,175.26","$21,676.53"
1,B,$205.00,"$30,945.75","$19,938.89"
2,C,$44.29,"$28,571.43","$18,000.37"
3,D,$137.52,"$22,649.62","$13,098.89"
4,E,$128.06,"$18,404.26","$7,887.42"
5,L,"$1,399.66","$42,944.79","$28,511.68"
6,S,$87.83,"$38,888.89","$21,086.86"
7,general,$44.29,"$34,642.86","$16,812.72"
